In [ ]:
'''
CREATE TABLE ko_game (
    gameKey INT AUTO_INCREMENT,
    gameid INT,
    date Date,
    homeKey INT,
    homeTeam VARCHAR(50),
    homeid INT,
    awayKey INT,
    awayTeam VARCHAR(50),
    awayid INT,
    homeScore INT,
    awayScore INT,

    PRIMARY KEY (gameKey)
)

'''

{109: 1, 144: 2, 110: 3, 111: 4, 112: 5, 145: 6, 113: 7, 114: 8, 115: 9, 116: 10, 117: 11, 118: 12, 108: 13, 119: 14, 146: 15, 158: 16, 142: 17, 121: 18, 147: 19, 133: 20, 143: 21, 134: 22, 135: 23, 137: 24, 136: 25, 138: 26, 139: 27, 140: 28, 141: 29, 120: 30}


In [2]:
import statsapi
sched = statsapi.schedule(start_date='01/01/2024',end_date='12/30/2024')
# team id key dict
t_info = {}

from team import team
t = team()
t.getAll()
for tt in t.data:
    t_info[tt["teamid"]] = tt['teamKey']
    
game_list = []
for i in sched:
    if i['status'] == 'Final' and i['game_type'] == "R":
        game = {}
        game['gameid'] = i['game_id']
        game['date'] = i['game_date']
        game['awayTeam'] = i['away_name']
        game['homeTeam'] = i['home_name']
        game['awayKey'] = t_info[i['away_id']]
        game['homeKey'] = t_info[i['home_id']]
        game['awayid'] = i['away_id']
        game['homeid'] = i['home_id']
        game['homeScore'] = i['home_score']
        game['awayScore'] = i['away_score']
        game_list.append(game)
        # print(i)



In [3]:
from game import game
g = game()
g.truncate()
count = 0
sql = f"INSERT INTO `{g.tn}` ("
for field in g.fields:
    sql += f"`{field}`,"
    count +=1
sql = sql[0:-1] + ') VALUES ('
tokens = ("%s," * count)[0:-1]
sql += tokens + ');'
print(sql)
n = 0
vals = []
for g_data in game_list:
    val = []
    for field in g.fields:
        val.append(g_data[field])
    vals.append(val)
    if n % 50 == 0:
        g.cur.executemany(sql,vals)
        vals = []
if len(vals) > 0:
    g.cur.executemany(sql,vals)
# print(sql,vals)


INSERT INTO `ko_game` (`gameid`,`date`,`homeKey`,`homeTeam`,`homeid`,`awayKey`,`awayTeam`,`awayid`,`homeScore`,`awayScore`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);


In [ ]:
'''
SELECT *, ko_team.teamKey
FROM ko_game
JOIN ko_team 
ON ko_game.homeid=ko_team.teamid;
'''